<a href="https://colab.research.google.com/github/sudeepbhagat11/Fake-News-Classifier-LSTM/blob/main/Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [9]:
df = pd.read_csv('/content/WELFake_Dataset.csv')

In [10]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [11]:
df.isnull().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [12]:
df=df.dropna()

In [13]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1
5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...,1


In [17]:
X = df.drop('label', axis=1)
y = df['label']

In [21]:
X.shape

(71537, 3)

In [22]:
y.shape

(71537,)

In [24]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [25]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [26]:
voc_size = 5000

## Onehot Representation

In [27]:
messages=X.copy()

In [36]:
messages['title']

0        LAW ENFORCEMENT ON HIGH ALERT Following Threat...
2        UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3        Bobby Jindal, raised Hindu, uses story of Chri...
4        SATAN 2: Russia unvelis an image of its terrif...
5        About Time! Christian Group Sues Amazon and SP...
                               ...                        
72129    Russians steal research on Trump in hack of U....
72130     WATCH: Giuliani Demands That Democrats Apolog...
72131    Migrants Refuse To Leave Train At Refugee Camp...
72132    Trump tussle gives unpopular Mexican leader mu...
72133    Goldman Sachs Endorses Hillary Clinton For Pre...
Name: title, Length: 71537, dtype: object

In [37]:
# managing the indexes above there is 0 2 but 1 is missing
messages.reset_index(inplace=True)

In [38]:
messages

,index,Unnamed: 0,title,text
0,0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
1,2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ..."
2,3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
3,4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
4,5,5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...
...,...,...,...,...
71532,72129,72129,Russians steal research on Trump in hack of U....,WASHINGTON (Reuters) - Hackers believed to be ...
71533,72130,72130,WATCH: Giuliani Demands That Democrats Apolog...,"You know, because in fantasyland Republicans n..."
71534,72131,72131,Migrants Refuse To Leave Train At Refugee Camp...,Migrants Refuse To Leave Train At Refugee Camp...
71535,72132,72132,Trump tussle gives unpopular Mexican leader mu...,MEXICO CITY (Reuters) - Donald Trump’s combati...


In [39]:
import nltk
import re
from nltk.corpus import stopwords

In [34]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [40]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [42]:
corpus

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video',
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video',
 'bobbi jindal rais hindu use stori christian convers woo evangel potenti bid',
 'satan russia unv imag terrifi new supernuk western world take notic',
 'time christian group sue amazon splc design hate group',
 'dr ben carson target ir never audit spoke nation prayer breakfast',
 'hous intel chair trump russia fake stori evid anyth video',
 'sport bar owner ban nfl game show true american sport like speak rural america video',
 'latest pipelin leak underscor danger dakota access pipelin',
 'gop senat smack punchabl alt right nazi internet',
 'may brexit offer would hurt cost eu citizen eu parliament',
 'schumer call trump appoint offici overse puerto rico relief',
 'watch hilari ad call question health age clinton crime famili boss',
 'chang expect espn polit agenda despit huge subscrib declin breitbart'

In [41]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[255, 197, 2853, 4318, 822, 2961, 4530, 4733, 4477, 4384, 1733, 4208],
 [7,
  2182,
  1689,
  4220,
  2032,
  2742,
  2251,
  2278,
  4170,
  2416,
  3645,
  3265,
  2202,
  4208],
 [1315, 3142, 1744, 4096, 4786, 401, 2388, 2985, 3769, 547, 1850, 2534],
 [1479, 830, 512, 2273, 871, 4397, 957, 3871, 4998, 3461, 4446],
 [2843, 2388, 1059, 1128, 3459, 1515, 2597, 4904, 1059],
 [2488, 4425, 2166, 4597, 1223, 2701, 1342, 1509, 2647, 767, 4144],
 [1276, 3773, 3874, 264, 830, 1754, 401, 650, 4904, 4208],
 [4435,
  107,
  2160,
  2653,
  1828,
  3787,
  3553,
  4748,
  824,
  4435,
  4704,
  4430,
  697,
  76,
  4208],
 [912, 771, 17, 2339, 3139, 3407, 1219, 771],
 [276, 1422, 4828, 4697, 1125, 2279, 3888, 3657],
 [4979, 2310, 3137, 347, 1204, 3624, 2135, 3703, 2135, 1945],
 [754, 4306, 264, 2180, 1533, 2837, 1301, 462, 3071],
 [2155, 2468, 2597, 4306, 4358, 342, 3736, 3583, 180, 1033, 1643],
 [4984, 651, 3197, 2776, 1394, 2488, 4430, 1380, 1529, 172],
 [2754, 3062, 3513, 2652, 1801, 1276, 39

## Embedding Representation


In [43]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[ 255  197 2853 ...    0    0    0]
 [   7 2182 1689 ...    0    0    0]
 [1315 3142 1744 ...    0    0    0]
 ...
 [1349 2842 2210 ...    0    0    0]
 [ 264 4321 2004 ...    0    0    0]
 [3078 1302  168 ...    0    0    0]]


In [44]:
embedded_docs[1]

array([   7, 2182, 1689, 4220, 2032, 2742, 2251, 2278, 4170, 2416, 3645,
       3265, 2202, 4208,    0,    0,    0,    0,    0,    0], dtype=int32)

In [71]:
# Creating Model
embedded_vector_features = 40  # feature representation
model = Sequential()
model.add(Embedding(voc_size, embedded_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_4 (LSTM)               (None, 100)               56400     
                                                                 
 dense_4 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [72]:
len(embedded_docs),y.shape

(71537, (71537,))

In [73]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [74]:
X_final.shape,y_final.shape

((71537, 20), (71537,))

In [75]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [69]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
749/749 [==============================] - 17s 21ms/step - loss: 0.3318 - accuracy: 0.8515 - val_loss: 0.2877 - val_accuracy: 0.8767
Epoch 2/10
749/749 [==============================] - 5s 7ms/step - loss: 0.2359 - accuracy: 0.9045 - val_loss: 0.2583 - val_accuracy: 0.8897
Epoch 3/10
749/749 [==============================] - 6s 8ms/step - loss: 0.2064 - accuracy: 0.9171 - val_loss: 0.2528 - val_accuracy: 0.8969
Epoch 4/10
749/749 [==============================] - 5s 7ms/step - loss: 0.1874 - accuracy: 0.9270 - val_loss: 0.2574 - val_accuracy: 0.8970
Epoch 5/10
749/749 [==============================] - 6s 9ms/step - loss: 0.1647 - accuracy: 0.9368 - val_loss: 0.2601 - val_accuracy: 0.8963
Epoch 6/10
749/749 [==============================] - 6s 7ms/step - loss: 0.1404 - accuracy: 0.9453 - val_loss: 0.3045 - val_accuracy: 0.8891
Epoch 7/10
749/749 [==============================] - 5s 6ms/step - loss: 0.1205 - accuracy: 0.9537 - val_loss: 0.3015 - val_accuracy: 0.8912
Epoc

## Adding Dropout

In [83]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 20, 40)            200000    
                                                                 
 dropout_11 (Dropout)        (None, 20, 40)            0         
                                                                 
 lstm_10 (LSTM)              (None, 100)               56400     
                                                                 
 dropout_12 (Dropout)        (None, 100)               0         
                                                                 
 dense_12 (Dense)            (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [84]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
749/749 [==============================] - 18s 19ms/step - loss: 0.3365 - accuracy: 0.8458 - val_loss: 0.2656 - val_accuracy: 0.8901
Epoch 2/10
749/749 [==============================] - 6s 8ms/step - loss: 0.2459 - accuracy: 0.8993 - val_loss: 0.2592 - val_accuracy: 0.8910
Epoch 3/10
749/749 [==============================] - 6s 7ms/step - loss: 0.2190 - accuracy: 0.9120 - val_loss: 0.2527 - val_accuracy: 0.8970
Epoch 4/10
749/749 [==============================] - 7s 9ms/step - loss: 0.2030 - accuracy: 0.9200 - val_loss: 0.2499 - val_accuracy: 0.8963
Epoch 5/10
749/749 [==============================] - 5s 7ms/step - loss: 0.1874 - accuracy: 0.9253 - val_loss: 0.2704 - val_accuracy: 0.8897
Epoch 6/10
749/749 [==============================] - 6s 8ms/step - loss: 0.1699 - accuracy: 0.9323 - val_loss: 0.2696 - val_accuracy: 0.8944
Epoch 7/10
749/749 [==============================] - 6s 8ms/step - loss: 0.1522 - accuracy: 0.9400 - val_loss: 0.2639 - val_accuracy: 0.8934
Epoc

# Performance Metrices and Accuracy

In [85]:
y_pred=model.predict(X_test)

738/738 [==============================] - 2s 2ms/step


In [86]:
y_pred = np.where(y_pred > 0.5,1,0)  ##AUC ROC Curve

In [87]:
from sklearn.metrics import confusion_matrix

In [88]:
confusion_matrix(y_test,y_pred)

array([[10560,  1133],
       [ 1385, 10530]])

In [89]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8933412402575398

In [90]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.88      0.90      0.89     11693
           1       0.90      0.88      0.89     11915

    accuracy                           0.89     23608
   macro avg       0.89      0.89      0.89     23608
weighted avg       0.89      0.89      0.89     23608

